#### Note: 

- Slate is defined as that day of daily fantasy competitions, pretty much synonymous with date --> slate is all games played that day
- For the time being, this dataset will not accomodate unusual schedules as it does not include start times for games, so it will just include all games played that day.
    - `slate_size` is going to be a column to refer to how many games were played on a given day, i.e. the number of teams for each day divided by 2
    - This will be useful to analyze patterns in optimal lineups, for example:
        - How many players from a team / game would one want to include in their 9-player lineup
        - Is it better to focus on certain games and ignore others, or is it better to try and have the best players from each team
        - How much of an impact does `total_pts` and `diff_score` have on fpts outcomes

In [ ]:
import pandas as pd
import numpy as np

# pandas settings....

from file_manager import FileManager
fm = FileManager()

In [ ]:
pd.options.display.max_rows = 200

In [ ]:
def categorizing_slates(**kwargs):
    df: pd.DataFrame = fm.load_clean_data()

    ret = (df
           .drop_duplicates(['date', 'team'])
           .groupby('date')
           ['team']
           .agg(['count'])
          )
    
    slate_sizes = {date: ret.loc[date, 'count'] // 2 for date in ret.index}
    
    for date, size in slate_sizes.items():
        df.loc[df['date']==date, 'slate_size'] = size
    
    df['slate_size'] = df['slate_size'].astype('uint8')
    
    # Categorize slates by quartile, excluding slates which include single game as that is different format
    
    df: pd.DataFrame = df.loc[df['slate_size']>1]
    
    slate_descrip = df['slate_size'].describe()
    
    slate_nums_categories = {
        tuple(range(*(int(slate_descrip['min']), int(slate_descrip['25%'])))): 1, # first element will always be 2
        tuple(range(*(int(slate_descrip['25%']), int(slate_descrip['50%'])))): 2,
        tuple(range(*(int(slate_descrip['50%']), int(slate_descrip['75%'])))): 3,
        tuple(range(*(int(slate_descrip['75%']), int(slate_descrip['max'])+1))): 4,
        
    }
    
    for nums, cat in slate_nums_categories.items():
        df.loc[ df['slate_size'].isin(nums), 'slate_cat' ] = cat
    
    df['slate_cat'] = df['slate_cat'].astype('uint8')
    
#     fm.save_dataframe(df, 'season-data-slates')
    return df
#     return None

In [ ]:
def sort_matchups(**kwargs):
    
    # Add slate categories
    df: pd.DataFrame = categorizing_slates()
    
    
    # team fantasy points
    tm = (df
          .groupby(['date', 'team'])
          ['fpts']
          .agg([np.sum])
         )
    
    msg = 'Adding team fantasy points\n'
    print(msg)
    for idx in tqdm(tm.index):
        date, team = idx
        df.loc[(df['date']==date) & (df['team']==team), 'team_fpts'] = tm.loc[idx,'sum']
    
    
    df['matchup'] = tuple(zip(df['team'], df['opp']))
    df['matchup'] = df['matchup'].map(lambda x: tuple(sorted(x)))
    
    # matchup fantasy points
    mu = (df
          .groupby(['date', 'matchup'])
          ['fpts']
          .agg([np.sum])
          .reset_index()
         )
    
#   Doing it this way because issue with having nested tuple in the index

    matchup_info = {
        date: {
            matchup: (mu
                      .loc[(mu['date']==date)
                           & (mu['matchup']==matchup)
                          ]
                      ['sum']
                      .item()
                     )
            for matchup in mu.loc[mu['date']==date, 'matchup'].drop_duplicates()
        }
        for date in mu['date'].drop_duplicates()
    }
    
    print(msg.replace('team', 'matchup'))
    for date, mu_totals in tqdm(matchup_info.items()):
        for matchup, total in mu_totals.items():
            df.loc[(df['date']==date) & (df['matchup']==matchup), 'mu_fpts'] = total
            
    fm.save_dataframe(df, 'season-data-added-info')
    
    return df

In [ ]:
df = sort_matchups()

In [ ]:
current_teams: list[str,...] = (
    'MIL',
    'OKC',
    'CHA',
    'LAC',
    'ATL',
    'PHO',
    'SAC',
    'CLE',
    'MIA',
    'MIN',
    'NY',
    'WAS',
    'GS',
    'CHI',
    'BKN',
    'HOU'
]

In [ ]:
make_boxplots(teams=current_teams);

In [ ]:
# Versus these teams
make_boxplots(
    team_col='opp',
    teams=current_teams
);

In [ ]:
# Total fpts in games involving teams
make_boxplots(
    fpts_col='mu_fpts',
    teams=current_teams
);